# Creating a Simple Agent with Tracing

In [2]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [3]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [4]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out nutritional advice.
    You give concise answers.
    """
    )

Let's execute the Agent:

In [5]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are generally healthy and convenient. Key points:
    
    - Good nutrients: potassium, vitamin B6, vitamin C, fiber, and some vitamin A and magnesium.
    - Benefits: supports heart health, digestion (fiber and resistant starch in unripe bananas), quick energy from natural sugars.
    - Considerations: moderate sugar content; portion control if watching blood sugar. Unripe bananas have more resistant starch (lower GI) than ripe ones.
    - Limits: very high intake could contribute excess calories; people with kidney disease may need to limit potassium; allergies are rare but possible.
    
    A typical medium banana is a healthy, balanced snack or add-in. If diabetic, pair with protein/fat to slow sugar rise.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [6]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are quite healthy for most people.

Pros:
- Good source of potassium, vitamin B6, vitamin C, and 3 g+ fiber per medium banana.
- Low in fat, convenient, and provide quick energy.

Cons:
- Contain natural sugars; portion matters if you’re watching sugar intake or blood sugar.

Tips:
- One medium banana fits well into a balanced diet most days.
- If you want more fiber and slower sugar release, choose slightly underripe (green) bananas.
- people with kidney disease should monitor potassium intake.

Overall: a nutritious, portable fruit to include regularly in moderation.

_Good Job!_